In [ ]:
from typing import Optional
import model


## Basic BTW Model

In this section, we explore the basic BTW model to verify the correctness of 
implementation.

In [ ]:
graph = model.make_grid_graph(5, 5)
model = model.Model()


